In [1]:
import pandas as pd

In [2]:
# read in original production data from csv file
orig_data = pd.read_csv('monthly-production.csv')

In [3]:
# read in format for target Excel sheet
target_format = pd.read_csv('prod-by-operated-day.csv')

In [4]:
target_format.head()

,api,cum_gor_180,cum_gor_30,cum_gor_365,cum_gor_545,cum_gor_730,cum_gor_90,cum_oil_180,cum_oil_30,cum_oil_365,...,cum_water_365,cum_water_545,cum_water_730,cum_water_90,cum_wc_180,cum_wc_30,cum_wc_365,cum_wc_545,cum_wc_730,cum_wc_90
0,33053032160000,1194.321,1190.631,1205.677,1275.677,1329.975,1180.290,169683.0,31936.0,262305.0,...,46855.0,58148.0,66312.0,28376.0,17.605,32.070,15.156,14.625,14.371,21.395
1,33053036170000,1017.419,845.320,1205.925,1316.548,1399.867,968.606,93461.0,23067.0,151827.0,...,27718.0,33426.0,38146.0,16756.0,19.486,23.302,15.438,15.076,15.188,20.949
2,33053050680000,938.340,1730.823,NaN,NaN,NaN,1622.325,91096.0,14392.0,NaN,...,NaN,NaN,NaN,3483.0,18.487,NaN,NaN,NaN,NaN,10.609
3,33025024960000,NaN,1557.266,NaN,NaN,NaN,1409.905,NaN,15786.0,NaN,...,NaN,NaN,NaN,18878.0,NaN,38.552,NaN,NaN,NaN,37.823
4,33053051180000,1532.590,NaN,NaN,NaN,NaN,1025.867,89874.0,14136.0,NaN,...,NaN,NaN,NaN,2853.0,9.132,0.049,NaN,NaN,NaN,6.892


In [5]:
# check column names of target Excel file format
target_format.columns

Index(['api', 'cum_gor_180', 'cum_gor_30', 'cum_gor_365', 'cum_gor_545',
       'cum_gor_730', 'cum_gor_90', 'cum_oil_180', 'cum_oil_30', 'cum_oil_365',
       'cum_oil_545', 'cum_oil_730', 'cum_oil_90', 'cum_water_180',
       'cum_water_30', 'cum_water_365', 'cum_water_545', 'cum_water_730',
       'cum_water_90', 'cum_wc_180', 'cum_wc_30', 'cum_wc_365', 'cum_wc_545',
       'cum_wc_730', 'cum_wc_90'],
      dtype='object')

In [6]:
# check column names of original data
orig_data.columns

Index(['API', 'Pool_Nm', 'RPT_DATE', 'DAYS_PROD', 'BBLS_OIL_COND', 'OIL_RUNS',
       'BBLS_WTR', 'MCF_GAS', 'MCF_SOLD', 'FLARED', 'VENTED', 'Dt_Treat'],
      dtype='object')

In [7]:
# check how original data looks
orig_data.head()

,API,Pool_Nm,RPT_DATE,DAYS_PROD,BBLS_OIL_COND,OIL_RUNS,BBLS_WTR,MCF_GAS,MCF_SOLD,FLARED,VENTED,Dt_Treat
0,33007001030000,MADISON,2009-01-01 00:00:00,31,705,704,5206,51,24,0,0,2010-12-16 00:00:00
1,33007001030000,MADISON,2009-02-01 00:00:00,28,597,597,4603,46,25,0,0,2010-12-16 00:00:00
2,33007001030000,MADISON,2009-03-01 00:00:00,31,701,704,5014,61,33,0,0,2010-12-16 00:00:00
3,33007001030000,MADISON,2009-04-01 00:00:00,30,690,687,4853,65,37,0,0,2010-12-16 00:00:00
4,33007001030000,MADISON,2009-05-01 00:00:00,31,734,736,5116,58,32,0,0,2010-12-16 00:00:00


In [8]:
# group original data by well API
grouped_data = orig_data.groupby(['API'])

In [9]:
# define new dataframe for cumulative results
cum_results = pd.DataFrame(columns=target_format.columns)
cum_results['api'] = orig_data.API.unique()
cum_results.set_index('api',inplace=True)

In [10]:
# define reporting intervals, based on target Excel file
report_intervals = [30,90,180,365,545,730]

In [11]:
# calculate cumulative oil values by well, for each interval
for well_api in cum_results.index:
    # get current well data as a data frame
    cur_well_data = grouped_data.get_group(well_api)
    # find max operating time of the current well
    max_cur_days = sum(cur_well_data.DAYS_PROD)
    
    # fill in values for each reporting interval
    for interval in report_intervals:
        cur_col_name = ['cum_oil_'+ str(interval)]
        cum_oil = 0
        cum_days = 0
        ind = 0
        if (interval <= max_cur_days): # takes care of cases where wells haven't been operating the whole time
            # find cumulative values for number of days reported
            for days in cur_well_data.DAYS_PROD.values:
                # get amount of oil produced in current month
                cur_oil = cur_well_data.iloc[ind]['BBLS_OIL_COND']
                # find rate of oil production in current month
                cur_rate = cur_oil/days
                ind += 1
                if (cum_days + days >= interval):
                    extra_days = interval - cum_days
                    # get oil produced just for the number of days required
                    cum_oil = cum_oil + extra_days*cur_rate
                    cum_results.loc[well_api][cur_col_name] = cum_oil
                    break
                else:
                    # if required interval has not been reached, add to cum_days and cum_oil
                    cum_days = cum_days + days
                    cum_oil = cum_oil + cur_oil
                
                    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in long_scalars


In [12]:
# checking values for cumulative oil columns
cum_results[['cum_oil_180', 'cum_oil_30', 'cum_oil_365',
       'cum_oil_545', 'cum_oil_730', 'cum_oil_90']].head(20)

,cum_oil_180,cum_oil_30,cum_oil_365,cum_oil_545,cum_oil_730,cum_oil_90
api,,,,,,
33007001030000,4093.03,682.258,7991,11351.2,14042.6,2003
33007003020000,NaN,NaN,NaN,NaN,NaN,NaN
33007005040000,NaN,NaN,NaN,NaN,NaN,NaN
33007010000000,NaN,26.129,NaN,NaN,NaN,418.2
33007011110000,1373,299.161,2064.71,2135,2140,751.556
33007011160000,797,42.5806,1587.68,2284,3129,462
33007011480000,854.333,156.774,1192,2901.61,5431.65,492
33007012030000,22390.1,5610.73,35547.8,45655.5,54129.2,14038.9
33007012540000,2021.48,21.2903,4129,5816.4,6534.77,812


In [13]:
# calculate cumulative water by well, for each interval
for well_api in cum_results.index:
    # get current well data as a data frame
    cur_well_data = grouped_data.get_group(well_api)
    # find max operating time of the current well
    max_cur_days = sum(cur_well_data.DAYS_PROD)
    
    # fill in values for each reporting interval
    for interval in report_intervals:
        cur_col_name = ['cum_water_'+ str(interval)]
        cum_water = 0
        cum_days = 0
        ind = 0
        if (interval <= max_cur_days): # takes care of cases where wells haven't been operating the whole time
            # find cumulative values for number of days reported
            for days in cur_well_data.DAYS_PROD.values:
                # get amount of water produced in current month
                cur_water = cur_well_data.iloc[ind]['BBLS_WTR']
                # find rate of water production in current month
                cur_rate = cur_water/days
                ind += 1
                if (cum_days + days >= interval):
                    extra_days = interval - cum_days
                    # get oil produced just for the number of days required
                    cum_water = cum_water + extra_days*cur_rate
                    cum_results.loc[well_api][cur_col_name] = cum_water
                    break
                else:
                    # if required interval has not been reached, add to cum_days and cum_water
                    cum_days = cum_days + days
                    cum_water = cum_water + cur_water

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in long_scalars


In [14]:
# check columns of results for cumulative water
cum_results[['cum_water_180',
       'cum_water_30', 'cum_water_365', 'cum_water_545', 'cum_water_730',
       'cum_water_90']].head(20)

,cum_water_180,cum_water_30,cum_water_365,cum_water_545,cum_water_730,cum_water_90
api,,,,,,
33007001030000,29445.5,5038.06,59363,87372.3,120315,14823
33007003020000,NaN,NaN,NaN,NaN,NaN,NaN
33007005040000,NaN,NaN,NaN,NaN,NaN,NaN
33007010000000,NaN,1416.77,NaN,NaN,NaN,4541.4
33007011110000,5371,327.484,13612.2,18193.2,18756,1224.11
33007011160000,3594,201.29,6475.84,9820.77,12845,2146
33007011480000,2242.1,408.387,4153,11775.8,20606.2,1197
33007012030000,5204.55,2261.93,7583.71,9483.23,11071.7,3873.23
33007012540000,6978.16,78.3871,14889,18046,19332.8,2972


In [15]:
# calculate water cut by well, for each interval
for well_api in cum_results.index:
    # get current well data as a data frame
    cur_well_data = grouped_data.get_group(well_api)
    # find max operating time of the current well
    max_cur_days = sum(cur_well_data.DAYS_PROD)
    
    for interval in report_intervals:
        cur_col_name = ['cum_wc_'+ str(interval)]
        cur_wc = 0
        cum_days = 0
        ind = 0
        if (interval <= max_cur_days): # takes care of cases where wells haven't been operating the whole time
            # find cumulative values for number of days reported
            for days in cur_well_data.DAYS_PROD.values:
                bbls_water = cur_well_data.iloc[ind]['BBLS_WTR']
                bbls_oil = cur_well_data.iloc[ind]['BBLS_OIL_COND']
                cur_wc = bbls_water*100/(bbls_water+bbls_oil)
                ind += 1
                if (cum_days >= interval):
                    cum_results.loc[well_api][cur_col_name] = cur_wc
                    break
                else:
                    # if required interval has not been reached, add to cum_days
                    cum_days = cum_days + days

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in long_scalars


In [ ]:
# check cumulative water values
cum_results[['cum_wc_180', 'cum_wc_30', 'cum_wc_365', 'cum_wc_545',
       'cum_wc_730', 'cum_wc_90']].head(20)

In [ ]:
# calculate gas oil ratio by well, for each interval
for well_api in cum_results.index:
    # get current well data as a data frame
    cur_well_data = grouped_data.get_group(well_api)
    # find max operating time of the current well
    max_cur_days = sum(cur_well_data.DAYS_PROD)
    
    for interval in report_intervals:
        cur_col_name = ['cum_gor_'+ str(interval)]
        cur_gor = 0
        cum_days = 0
        ind = 0
        if (interval <= max_cur_days): # takes care of cases where wells haven't been operating the whole time
            # find cumulative values for number of days reported
            for days in cur_well_data.DAYS_PROD.values:
                vol_gas = cur_well_data.iloc[ind]['MCF_GAS']#*1e6
                vol_oil = cur_well_data.iloc[ind]['BBLS_OIL_COND']#*5.61458333
                cur_gor = vol_gas/vol_oil
                ind += 1
                if (cum_days >= interval):
                    cum_results.loc[well_api][cur_col_name] = cur_gor
                    break
                else:
                    cum_days = cum_days + days

In [ ]:
cum_results[['cum_gor_180', 'cum_gor_30', 'cum_gor_365', 'cum_gor_545',
       'cum_gor_730', 'cum_gor_90']].head(20)